In [70]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn import metrics


In [2]:
sales_df = pd.read_csv(r'C:\Users\Ashish\Desktop\machine learning\Advertising.csv')
sales_df.head()

,Unnamed: 0,TV,Radio,Newspaper,Sales
0,1,230.1,37.8,69.2,22.1
1,2,44.5,39.3,45.1,10.4
2,3,17.2,45.9,69.3,9.3
3,4,151.5,41.3,58.5,18.5
4,5,180.8,10.8,58.4,12.9


In [3]:
X = sales_df[['TV', 'Radio', 'Newspaper']]
Y = sales_df['Sales']

In [4]:
Y = np.array((Y-Y.mean()) /Y.std())
X = X.apply (lambda rec: (rec - rec.mean()) / rec.std(), axis = 0)

In [5]:
import random

In [6]:
def initialize(dim):
    np.random.seed( seed = 42)
    random.seed(42)
    b = random.random()

    w = np.random.rand(dim)
    return b,w

In [7]:
b , w = initialize(3)
print("Bias  : ", b, "Weights : ", w )

Bias  :  0.6394267984578837 Weights :  [0.37454012 0.95071431 0.73199394]


In [8]:
def predict_Y(b, w ,X):
    return b+ np.matmul(X, w)

In [9]:
b,w = initialize(3)
Y_hat = predict_Y( b, w, X)
Y_hat[0:10]

0    3.231496
1    1.707849
2    2.824761
3    2.753090
4    0.924486
5    3.171365
6    0.622344
7   -0.349354
8   -2.313095
9   -0.768030
dtype: float64

In [10]:
import math
def get_cost(Y, Y_hat):
    Y_resid = Y- Y_hat
    return np.sum(np.matmul(Y_resid.T , Y_resid)) / len(Y_resid)

In [11]:
b, w = initialize (3)
Y_hat = predict_Y(b, w, X)
get_cost( Y, Y_hat)

1.53031001985059

In [12]:
def update_beta(X, Y,Y_hat, b_0, w_0, learning_rate):
    db = (np.sum(Y_hat - Y)*2) /len(Y)
    dw = (np.dot((Y_hat - Y), X )*2) /len(Y)
    b_1 = b_0 - learning_rate *db
    w_1 = w_0 - learning_rate *dw
    return b_1, w_1

In [13]:
b, w = initialize(3)
print("After Initialize - Bias : ", b, "Weights :", w)
Y_hat = predict_Y(b, w, X)
b,w = update_beta(X, Y, Y_hat, b, w, 0.01)
print("After first update - Bias:", b, "Weights :", w)

After Initialize - Bias :  0.6394267984578837 Weights : [0.37454012 0.95071431 0.73199394]
After first update - Bias: 0.6266382624887261 Weights : [0.38079093 0.9376953  0.71484883]


In [14]:
def run_gradient_descent(X, Y, alpha =0.01, num_iterations = 100):
    b, w = initialize(X.shape[1])
    iter_num = 0
    gd_iteration_df = pd.DataFrame(columns = ['iteration', 'cost'])
    result_idx = 0
    for each_iter in range(num_iterations):
        Y_hat = predict_Y(b, w, X)
        this_cost = get_cost(Y, Y_hat)
        prev_b = b
        prev_w = w
        b, w = update_beta(X, Y, Y_hat, prev_b, prev_w, alpha)
        if(iter_num % 10 == 0):
            gd_iteration_df.loc[result_idx] = [iter_num, this_cost]
            result_idx +=1
        iter_num +=1
    print("Final estimate of b and w :", b, w)
    return gd_iteration_df, b, w

In [15]:
gd_iteration_df, b, w = run_gradient_descent(X, Y, alpha =0.001, num_iterations = 200)

Final estimate of b and w : 0.4284489581739146 [0.48270238 0.75265969 0.46109174]


In [16]:
gd_iteration_df[0:10]

,iteration,cost
0,0.0,1.530310
1,10.0,1.465201
2,20.0,1.403145
3,30.0,1.343996
4,40.0,1.287615
5,50.0,1.233868
6,60.0,1.182630
7,70.0,1.133780
8,80.0,1.087203
9,90.0,1.042793


In [36]:
X_train,X_test, Y_train, Y_test = train_test_split(sales_df[["TV", "Radio", "Newspaper"]], sales_df.Sales, test_size= 0.3, random_state = 42)

In [40]:
len(X_train)

140

In [42]:
len(X_test)

60

In [46]:
linreg = LinearRegression()
linreg.fit(X_train, Y_train)

LinearRegression()

In [48]:
linreg.intercept_

2.70894909251591

In [50]:
linreg.coef_

array([0.04405928, 0.1992875 , 0.00688245])

In [52]:
list(zip(["TV", "Radio", "Newspaper"], list(linreg.coef_)))

[('TV', 0.0440592809574652),
 ('Radio', 0.19928749689893954),
 ('Newspaper', 0.006882452222275458)]

In [62]:
#predicting the y value from the test set
Y_pred = linreg.predict(X_test)

In [68]:
#creating dataframe with 3 column named : actual,predicted and residuals 
#to store the respecitve valuse
test_pred_df= pd.DataFrame({'actual': Y_test, 'predicted': np.round(Y_pred,2), 'residuals': Y_test- Y_pred})
test_pred_df.sample(10)

,actual,predicted,residuals
126,6.6,11.15,-4.553147
170,8.4,7.35,1.049715
95,16.9,16.57,0.334604
195,7.6,5.22,2.375645
115,12.6,13.36,-0.755569
38,10.1,10.17,-0.070454
56,5.5,8.92,-3.415494
165,11.9,14.30,-2.402060
173,11.7,11.63,0.068431
9,10.6,12.18,-1.576049


In [72]:
r2= metrics.r2_score(Y_train, linreg.predict(X_train))
print("R squared", r2)

R squared 0.9055159502227753


In [74]:
mse= metrics.mean_squared_error(Y_test, Y_pred)

In [76]:
rmse= round(np.sqrt(mse),2)
print("RMSE", rmse)

RMSE 1.95
